In [2]:
import pandas as pd
import numpy as np

In [ ]:
# We just want the names. Let's use pandas to strip out only the names

In [3]:
df = pd.read_csv('cannabis.csv')

In [4]:
df.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [7]:
df.Strain.to_csv('strain_names.csv', index=False)

In [9]:
!head strain_names.csv

100-Og
98-White-Widow
1024
13-Dawgs
24K-Gold
3-Bears-Og
3-Kings
303-Og
3D-Cbd
3X-Crazy


In [15]:
# get a single long string of strain names separated by newline chars
with open('strain_names.csv') as f:
    file = f.read()

In [22]:
file[:100]

'100-Og\n98-White-Widow\n1024\n13-Dawgs\n24K-Gold\n3-Bears-Og\n3-Kings\n303-Og\n3D-Cbd\n3X-Crazy\n3Rd-Coast-Pan'

In [16]:
file_len = len(file)

In [17]:
file_len

29372

In [23]:
# Now following the PyTorch Example

In [52]:
import random
import string
import re

all_characters = string.printable
n_characters = len(all_characters)

In [50]:
chunk_len = 250


def random_chunk():
    '''filelen is length of chars in file'''
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

In [48]:
print(random_chunk())

-47
Ak-48
Aberdeen
Abusive-Og
Acapulco-Gold
Ace-Killer-Og
Ace-Of-Spades
Aceh
Acid
Acid-Dough
Afcrack
Afghan-Big-Bud
Afghan-Cow
Afghan-Diesel
Afghan-Hawaiian
Afghan-Haze
Afghan-Kush
Afghan-Skunk
Afghan-Sour-Kush
Afghan-Widow
Afghani
Afgahni-Bullrider
A


In [ ]:
## Build the Model

In [24]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        input = self.encoder(input.view(1,-1))
        output, hidden = self.gru(input.view(1,1,-1) ,hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
    


In [49]:
## Inputs and Targets

In [53]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

Variable containing:
 10
 11
 12
 39
 40
 41
[torch.LongTensor of size 6]



In [54]:
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target


In [55]:
# Evaluating

In [62]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str
    
    for p in range(len(prime_str)-1):
        _,hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)
        
    return predicted

In [57]:
# Training

In [58]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m,s)

In [59]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss=0
    
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])
        
    loss.backward()
    decoder_optimizer.step()
    
    return loss.data[0] / chunk_len

In [63]:
n_epochs = 2000
print_every = 100
plot_every=10
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses=[]
loss_avg = 0

for epoch in range(1, n_epochs+1):
    loss = train(*random_training_set())
    loss_avg += loss
     
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

 29s (100 5%) 2.4270]
Whica-Og
Loongherk-The
Sour-Fush
Chon-oonoka
Dam-Kush
Ian-Tigh-Dtur
Dolierers
Trruse
Ari-Haze
Boubbar- 

 4s (200 10%) 2.6515]
Whrenram
Veate-Sunk
Vre-Bed
Liese-Emand
Whee
Songaiin
ilut-4
Jack-47
Ick
Cherry
Siverl-Dreane
Cherry-H 

 37s (300 15%) 2.0116]
Whitim-Difer-Tanger
Goder-Finderder
Gurda-Kush
Porple-Chul
Chat-0ine
Far-Og
Bluebenber-Dack
Super-Bubb 

 8s (400 20%) 1.8203]
Whise
Og-Gaft
Penp-Kush
Klange-Tari
Og-Breck
Blue-Banger
Budack
Berry-Black
The-Og
PerSple-Jack
Jack-B 

 40s (500 25%) 2.0838]
White-Kush
Countie
Piesule-Cbd
White-Rock
Truit-Diens
White-Mist
Clue-Kush
Clemene
Chempan-Kush
Chur-K 

 13s (600 30%) 1.4640]
White
Purple-White
Diel-Sour-Dieles
Purple-Ansy-Purks
Purple-Pope
Shition-Alien-Alien-Og
Pine-Pon
Mumo 

 45s (700 35%) 1.5083]
White-Og
Mablist
Mad-Cookey
Cookie-Cookies
Purple-Cruit
Chooolate-Fut-Sheettor
Sour-Nerstht
Sugarral-A 

 17s (800 40%) 2.1547]
White-Diesel
Peall-Cherry
Hulper-Bubble
Sour-Bel-Tange
Super-Diesel
Super-Shem
Shark

In [74]:
print(evaluate('Em', 500, temperature=0.8))

Emai
Green-Kush
Grapefruit
Grean-Golden
Green-Golden
Green-Sour-Green-Darp
Galley-Grill
Oback-Dirt
Golden-Og
Golden-Trine
Green-Haze
Goldent-Og
Golden-Sour
Glelkelland-Godilly
Golden-Limeal
Golden-Forange
Golden-Haze
Goset-Goldent
Grunk-Durbal
Bedccry
Browbay
Blueberry-Kush
Bebla-78
Bega-Kush
Big-Hound
Hashberry-Riger
Haze-Og
Haze-Tangs-Banana
Sur-Is
Haze-Tream
Ormulange
Hawaiian-Beblue
Blueberry-Borth
Blueberry-Jonge-Og
Bedic-Jian
Black-Ladol-Bell-Sherbert
Black-Berry
Black-Candy
Blueberry-Medigh


In [70]:
print(evaluate('o', 500, temperature=0.9))

o-Golden
South-Beda
Solator-Haze
Sour-Shert
Sour-Bellito
Sour-Cookie
Sour-Cheese
Sour-Comoth
Northell-Kush
Obamd-Sour
Iced-Gold
Og-Tonic
Ing-Tangie
Goldon
Haze-Wally
Kush-Kush
Og-Wooms
Ire-Jofices
Green-Queen
Golden-Treane
Grapefruit-Kush
Gombilly
Sour-Indica
Goldent-Heam
Insconeo
Codfumed
Cherry-Cake
Sugar-Dream
Haze
Hawaiian-Tream
Headwollangie
Olmon-Tahin
Critical-Jack
Critical-Cherry-Og
Cali-Cake
Califorange-Og
Cat-Puncher
Cannadelic
Cookie-Kush
Calker-Og
Cannalel-Kush
Blonciame-Skunk
Bingo-B


In [66]:
print(evaluate('M', 200, temperature=0.9))

Medist
Clemitone
Cannadend-Og
lashtar
Coidega
Comonebalan
Caplerical
Madgol
Jacky-Crapefrion
Jaues-Grapidruit
Purple-Og
Platian-Pineapt
Trescd
Ogut
Tumbold
Tree-Bubbadg
Pineape-Snow
Platival-Ino
Orange


In [73]:
print(evaluate('1', 500, temperature=0.9))

1
Blactcry
Black-Haze
Blueberry-Wlegs
Blunchion
Black-Ind
Blackberry-Cherry-Diamond-Je
Blacky-Banner
Bickel-Kush
Blueberry-Dreamel
Bangie-Dark
Bobbruity-Cheese
Blueberry-Sklaze
JamaicanYJean
Jill-Kush
Jawaiian
Jawaiian-Jelone
Jack-Cream
Jure-Purple
JaJple
Jerry-Og
Ulavertonioth
The-Bubbacolitious
Tawgio
Tonickie-Bubba
The-Cough
Tangenime-Toria
Omega-Og
Hashberry
Hode-Dream
Haze
Haze-Og
Humbe-Skunk
Blueberry-Hly-Hood-Daddy
Blaund-Headumer-Cake
Blueberry-Tonic
Black-Dond-Mango-Deatical-Sour-Blook
C


In [72]:
print(evaluate('Z', 500, temperature=0.7))

ZSkunk
Sour-Cookie-Kush
Sour-Alien-Og
Sour-Calyx
Sour-Amnesiol
Sur-Queen
Sugar-Sumater
Skunk-47
Orange-Candy
Purple-Trainwreck
Purple-Trainerster
Trandwrape
Triganeda
Huels
Ogeger-Candy
Tangerinit
Traininus-Og
Tamemel-Kush
Traline-Haze
Tangerine-Haze
Trush-Train
Trinne-Trip
Tressinge-Train
Tuna-Critus
Biesel
Jack-Tangie
Golden-Gold
Hayama-Cheese
Headbangel
Golden-Calyx
Dowboo-Kush
Hawaiian-Dick
Black-Beat-Bubba
Blus-Bog
Bedicole
Blueberry-Og
Blueberry-Diesel
Brananuania
Bananal-Og
Banastar-Pie
Bl


In [75]:
print(evaluate('w', 500, temperature=0.66))

w-Train
Haze-Wreck
Hemmalladon
Doylanda
Sour-Diesel
Sour-Beall-Kush
Sour-Budch
Sweet-Sour
Double-Cheese
Sourhala
Sour-Dream
Sour-Cheese
Sour-Calyx
Sour-Pine
Swissillata
Sugar-Sour
Sour-Potter
Sour-Cherry-Sunshine
Sur-Sunshine
Sunshine-Dream
Calif-Kush
Califorstade-Og
Calicy-Kush
Californ-Cape
Cooth-Calif
Calix-Og
Candey-Og
Candy-Kush
Cannalif-Candy
Caramel-Og
Cannalife
Califrange-Cookies
Critical-Ple
Critical-Haze
Cannaly-Cheese
Critical-Kush
Caramelicheas-Candy
Cannalay-Beast-Og
Casher-Purple
Be
